In [2]:
from IPython.display import clear_output
%cd /content/drive/MyDrive/collab_sandbox/learn_DL/dive_into_dl/
# !pip install colabcode
clear_output()

# Pandas

https://pandas.pydata.org/docs/user_guide/10min.html#setting

# Titanic Kaggle

https://www.kaggle.com/c/titanic

In [19]:
import pandas as pd
import torch
import numpy as np
from torch.utils import data

In [10]:
dataset_train = pd.read_csv("./data/TitanicKaggle/train.csv")
dataset_test = pd.read_csv("./data/TitanicKaggle/test.csv")

drop_columns = ['Name', 'PassengerId', 'Ticket', 'Cabin']
dataset_train = dataset_train.drop(columns=drop_columns)

In [5]:
dataset_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [8]:
def encode_categorical(dataset, columns):
  dataset = dataset.copy(deep=True)
  dataset_cols = dataset.columns.values
  for column in columns:
    if column in dataset_cols:
      values_list = dataset[column].values.tolist()
      list_columns = sorted(list(map(str, list(set(values_list)))))
      list_columns_renamed = [f'{column}_{item}' for item in list_columns]
      dict_columns = {item: i for (i, item) in enumerate(list_columns)}
      
      for i, value in enumerate(values_list):
        value = str(value)

        ohe_hot = [0 for _ in range(len(list_columns))]
        ohe_hot[dict_columns.get(value)] = 1
        dataset.loc[i, list_columns_renamed] = ohe_hot
      dataset = dataset.drop(columns=[column],) 
  
  return dataset

# encode_categorical(dataset_train, ['Sex', 'Embarked']).head(3)

In [11]:
categorical_labels = ['Pclass', 'SibSp',  'Embarked', 'Sex', 'Parch']
dataset_train = encode_categorical(dataset_train, categorical_labels)
dataset_train.head(3)

,Survived,Age,Fare,Pclass_1,Pclass_2,Pclass_3,SibSp_0,SibSp_1,SibSp_2,SibSp_3,SibSp_4,SibSp_5,SibSp_8,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan,Sex_female,Sex_male,Parch_0,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6
0,0,22.0,7.2500,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,38.0,71.2833,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,26.0,7.9250,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
def standardize_data(dataset, exclude_columns):
  dataset = dataset.copy(deep=True)
  dataset_cols = list(set(dataset.columns.values) - set(exclude_columns))
  for column_name in dataset_cols:
    # column_name = "Age"
    column = dataset[column_name].to_numpy()
    nan_indexes = np.isnan(column)
    real_indexes = ~np.isnan(column)

    n = real_indexes.astype(int).sum()
    real_column = column[real_indexes]

    real_column_mean = real_column.sum() / n # or .mean()  
    real_column_var = (np.sum([(column[i]-real_column_mean)**2 if item else 0 for i, item in enumerate(list(real_indexes))]) / n)**(1/2)
    std_column = np.array([(column[i]-real_column_mean) / real_column_var if item else 0 for i, item in enumerate(list(real_indexes))])
    
    dataset.loc[:, column_name] = std_column
    # return dataset
    # break 
  return dataset

standardize_data(dataset_train, ['Survived'])

,Survived,Age,Fare,Pclass_1,Pclass_2,Pclass_3,SibSp_0,SibSp_1,SibSp_2,SibSp_3,SibSp_4,SibSp_5,SibSp_8,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan,Sex_female,Sex_male,Parch_0,Parch_1,Parch_2,Parch_3,Parch_4,Parch_5,Parch_6
0,0,-0.530377,-0.502445,-0.565685,-0.510152,0.902587,-1.465746,1.806421,-0.180125,-0.135225,-0.143592,-0.075122,-0.088986,-0.482043,-0.307562,0.619306,-0.047431,-0.737695,0.737695,0.560499,-0.390707,-0.314076,-0.075122,-0.067153,-0.075122,-0.03352
1,1,0.571831,0.786845,1.767767,-0.510152,-1.107926,-1.465746,1.806421,-0.180125,-0.135225,-0.143592,-0.075122,-0.088986,2.074505,-0.307562,-1.614710,-0.047431,1.355574,-1.355574,0.560499,-0.390707,-0.314076,-0.075122,-0.067153,-0.075122,-0.03352
2,1,-0.254825,-0.488854,-0.565685,-0.510152,0.902587,0.682247,-0.553581,-0.180125,-0.135225,-0.143592,-0.075122,-0.088986,-0.482043,-0.307562,0.619306,-0.047431,1.355574,-1.355574,0.560499,-0.390707,-0.314076,-0.075122,-0.067153,-0.075122,-0.03352
3,1,0.365167,0.420730,1.767767,-0.510152,-1.107926,-1.465746,1.806421,-0.180125,-0.135225,-0.143592,-0.075122,-0.088986,-0.482043,-0.307562,0.619306,-0.047431,1.355574,-1.355574,0.560499,-0.390707,-0.314076,-0.075122,-0.067153,-0.075122,-0.03352
4,0,0.365167,-0.486337,-0.565685,-0.510152,0.902587,0.682247,-0.553581,-0.180125,-0.135225,-0.143592,-0.075122,-0.088986,-0.482043,-0.307562,0.619306,-0.047431,-0.737695,0.737695,0.560499,-0.390707,-0.314076,-0.075122,-0.067153,-0.075122,-0.03352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,-0.185937,-0.386671,-0.565685,1.960202,-1.107926,0.682247,-0.553581,-0.180125,-0.135225,-0.143592,-0.075122,-0.088986,-0.482043,-0.307562,0.619306,-0.047431,-0.737695,0.737695,0.560499,-0.390707,-0.314076,-0.075122,-0.067153,-0.075122,-0.03352
887,1,-0.737041,-0.044381,1.767767,-0.510152,-1.107926,0.682247,-0.553581,-0.180125,-0.135225,-0.143592,-0.075122,-0.088986,-0.482043,-0.307562,0.619306,-0.047431,1.355574,-1.355574,0.560499,-0.390707,-0.314076,-0.075122,-0.067153,-0.075122,-0.03352
888,0,0.000000,-0.176263,-0.565685,-0.510152,0.902587,-1.465746,1.806421,-0.180125,-0.135225,-0.143592,-0.075122,-0.088986,-0.482043,-0.307562,0.619306,-0.047431,1.355574,-1.355574,-1.784124,-0.390707,3.183944,-0.075122,-0.067153,-0.075122,-0.03352
889,1,-0.254825,-0.044381,1.767767,-0.510152,-1.107926,0.682247,-0.553581,-0.180125,-0.135225,-0.143592,-0.075122,-0.088986,2.074505,-0.307562,-1.614710,-0.047431,-0.737695,0.737695,0.560499,-0.390707,-0.314076,-0.075122,-0.067153,-0.075122,-0.03352
